# 타이타닉 생존자 예측 데이터셋 (Titanic: Machine Learning from Disaster)

이 데이터셋은 1912년 침몰한 타이타닉호의 승객 정보를 담고 있습니다. 주요 목표는 승객의 여러 특성을 바탕으로 생존 여부(Survived)를 예측하는 것입니다.

#### 데이터 사전 (Data Dictionary)

| 변수명 (Variable) | 정의 (Definition) | 키 (Key) |
| :--- | :--- | :--- |
| **Survival** | 생존 여부 | 0 = No, 1 = Yes |
| **Pclass** | 티켓 등급 (사회-경제적 지위) | 1 = 1st, 2 = 2nd, 3 = 3rd |
| **Sex** | 성별 | |
| **Age** | 나이 | |
| **SibSp** | 타이타닉호에 동승한 형제/배우자 수 | |
| **Parch** | 타이타닉호에 동승한 부모/자녀 수 | |
| **Ticket** | 티켓 번호 | |
| **Fare** | 운임 (승객 요금) | |
| **Cabin** | 객실 번호 | |
| **Embarked** | 중간 정착 항구 | C = Cherbourg, Q = Queenstown, S = Southampton |

#### 변수 참고 사항 (Variable Notes)

* **pclass**: 사회-경제적 지위(SES)의 대리 지표
    * 1st = 상류층 (Upper)
    * 2nd = 중산층 (Middle)
    * 3rd = 하류층 (Lower)
* **age**: 나이가 1세 미만인 경우 분수(fractional)로 표시됩니다. 나이가 추정치인 경우 xx.5 형태로 표시됩니다.
* **sibsp**: 가족 관계를 다음과 같이 정의합니다.
    * Sibling: 형제, 자매, 의붓형제, 의붓자매
    * Spouse: 남편, 아내 (내연녀, 약혼자는 무시됨)
* **parch**: 가족 관계를 다음과 같이 정의합니다.
    * Parent: 어머니, 아버지
    * Child: 딸, 아들, 의붓딸, 의붓아들
    * 참고: 유모(nanny)와 함께 여행한 어린이는 parch=0으로 표시됩니다.

#### 1. 데이터셋 불러오기

In [ ]:
import pandas as pd
import numpy as np
import os

trainCSV = "C:\\Users\\Brain\\github\\DataSicence\\scikit-learn\\data\\titanic\\train.csv"

In [ ]:
# get Current Working Directory
HOME = os.getcwd()
trainCSV = os.path.join(HOME, "data\\titanic\\train.csv")
print(trainCSV)

train_df = pd.read_csv(trainCSV)
train_df.head()

#### 2. 데이터 확인 (info(), describe(), head(), tail())

In [ ]:
train_df.head()

In [ ]:
train_df.tail()

In [ ]:
train_df.info()

In [ ]:
train_df.describe()

#### 3. 결측치 확인

In [ ]:
# 데이터프레임 전체에서 컬럼별로 결측치 몇개씩 있는지 확인
train_df.isnull().sum()

#### 4. 결측치 처리(수치형, 문자형)

**중앙값(median)**

데이터 : [3, 1, 2, 4, 5, 100] -> 정렬 [1, 2, 3, 4, 5, 7, 100] -> 중앙값 4

**평균(mean)**

데이터 : [3, 1, 2, 4, 5, 100] -> 평균 21.666666666666668


In [ ]:
# fillna()를 사용하여 결측치 채우기 (평균값, 중앙값)
# train_df['Age']
# train_df['Age'].fillna(train_df['Age'].mean(), inplace=False)
train_df['Age'].fillna(train_df['Age'].median(), inplace=False)

In [ ]:
# sklearn의 simple imputer를 사용한 결측치 처리
from sklearn.impute import SimpleImputer

# 1. Simple imputer 생성
# strategy는 mean, median, most_frequent, constant
# 수치형 데이터는 4가지 모두 사용가능
# 문자열 데이터는 most_frequent, constant 사용
imputer = SimpleImputer(strategy='mean')

# 2. fit_transform으로 결측치 처리
train_df[['Age']] = imputer.fit_transform(train_df[['Age']])

In [ ]:
train_df.isnull().sum()

In [ ]:
# 최빈값이 무엇인지 확인
train_df['Embarked'].value_counts()

In [ ]:
# Embarked 결측치 처리 (imputer로 최빈값(most_frequent)으로 채우기)

# 1. imputer 생성
imputer = SimpleImputer(strategy='most_frequent')

# 2. fit_transform으로 결측치 처리
train_df[['Embarked']] = imputer.fit_transform(train_df[['Embarked']])

In [ ]:
train_df.isnull().sum()

In [ ]:
# 최빈값으로 데이터가 추가된 것을 확인
train_df['Embarked'].value_counts()

In [ ]:
# 'Cabin' 데이터가 존재하는 행만 선택
train_df[train_df['Cabin'].notnull()].head(5)

In [ ]:
# train_df['Cabin'] 컬럼은 삭제
train_df.drop(columns=['Cabin'], inplace=True)

In [ ]:
# 'Cabin'컬럼이 삭제된 것을 확인할 수 있다.
train_df.head()

#### 5. 레이블 인코딩
* 인공지능 모델은 문자열을 처리할 수 없습니다. 따라서 문자열 데이터를 숫자로 변환하는 것이 필요합니다.
* 이 과정을 레이블 인코딩이라고 합니다.

In [ ]:
# train_df에서 레이블 인코딩을 해야하는 대상은? ['Sex, 'Embarked']
# sklearn에는 LabelEncoder가 있다. 'male', 'Female' -> 0, 1
from sklearn.preprocessing import LabelEncoder

# 1. 레이블 인코더를 생성
label_encoder = LabelEncoder()
# Label_encoder2 = LabelEncoder()
# 2. fit_transform으로 레이블 인코딩을 적용
# fit은 데이터를 확인 'male', 'Female' -> 0, 1
train_df['Sex_lable'] = label_encoder.fit_transform(train_df['Sex'])
# fit은 데이터를 확인 'S', 'C', 'Q' -> 0, 1, 2
train_df['Embarked_lable'] = label_encoder.fit_transform(train_df['Embarked'])

In [ ]:
train_df.head()

#### 6. 원-핫 인코딩
원-핫 인코딩으로 변환하는 이유는 컴퓨터가 데이터를 0, 1, 2 라는 숫자형태 보다는 벡터 형태로 처리할 수 있도록 변환하는 것이 효율적입니다. (원-핫 벡터)

In [ ]:
# pandas에는 원-핫 인코딩을 위한 get_dummies가 있다.
# train_df에서 원-핫 인코딩을 해야하는 대상은? ['Pclass', 'Sex', 'Embarked']
pd.get_dummies(train_df, columns=['Pclass', 'Sex', 'Embarked'])

#### 7. 정규화(Normalization, 표준화 : Standardization)

In [ ]:
movie = {'naver': [2, 4, 6, 8, 10], 
         'netflix': [1, 2, 3, 4, 5]
         }

In [ ]:
movie = pd.DataFrame(data=movie)
movie

#### MinMaxScaler

##### 1. 같은 데이터인데 스케일이 서로 다를 때 스케일을 맞춰주는 역할

In [ ]:
# sklearn에서는 정규화를 위한 함수로 MinMaxScaler가 있다.
from sklearn.preprocessing import MinMaxScaler

# 1. MinMaxScaler를 생성 (min=0, max=5)
min_max_scaler = MinMaxScaler(feature_range=(0,5))

# 2. fit_transform으로 데이터를 변환
movie2 = min_max_scaler.fit_transform(movie)

In [ ]:
movie2

* 여러개의 컬럼이 존재할 때 서로 다른 데이터의 특성(단위)이 다른 데이터를 비교할 수 있게 해준다.

예시를 집값 데이터로 들면,
집의 크기(m^2), 방의 갯수(개), 지하철 역과의 거리(km), 마트와의 거리(km)
집의 크기 : 50 ~ 130
방의 갯수 : 1 ~ 5
지하철 역과의 거리 : 0.1 ~ 10
마트와의 거리 : 0.1 ~ 9

In [ ]:
# 내가 만든거
import pandas as pd
import numpy as np
# 데이터 생성을 위한 딕셔너리 정의
data = {
    '집의 크기(m^2)': [85, 120, 55, 90, 110],        # 50 ~ 130 범위
    '방의 갯수(개)': [3, 4, 1, 3, 5],                # 1 ~ 5 범위
    '지하철 역과의 거리(km)': [0.5, 3.2, 8.5, 1.2, 5.0], # 0.1 ~ 10 범위
    '마트와의 거리(km)': [0.2, 1.5, 7.8, 0.8, 4.2]      # 0.1 ~ 9 범위
}
# 데이터프레임 생성
house_df = pd.DataFrame(data)
# 데이터 확인
print("--- 생성된 집값 데이터 (정규화 전) ---")
display(house_df)
# 각 컬럼의 최소/최대값 확인 (스케일 차이 확인용)
print("\n--- 각 특성(Feature)의 데이터 범위 ---")
print(house_df.agg(['min', 'max']))

In [ ]:
# 강사님꺼
import pandas as pd

# 1. 데이터 생성 (딕셔너리 형태)
data = {
    '집의 크기(m2)': [85.5, 120.2, 55.0, 105.8, 72.4],
    '방의 갯수(개)': [3, 4, 1, 3, 2],
    '지하철역과의 거리(km)': [0.5, 2.1, 0.2, 5.4, 1.1],
    '마트와의 거리(km)': [1.2, 0.8, 4.5, 2.3, 0.3]
}

# 2. 데이터프레임 생성
df = pd.DataFrame(data)

# 3. 결과 출력
print(df)

In [ ]:
# df를 정규화(minMaxScaler)
# 1. MinMaxScaler를 생성 (min=0, max=1)
# 일반적으로 정규화시에는 0 ~ 1 사이 값으로 많이 사용함
min_max_scaler = MinMaxScaler(feature_range=(0,1))

# 2. fit_transform으로 데이터를 변환
df2 = min_max_scaler.fit_transform(df)

### 표준화 (Standard Scaling)

* 평균이 0이고, 표준 편차가 1이 되도록 변환

In [ ]:
# 표준화를 위한 샘플 데이터셋 생성
x = np.arange(10)

# 이상치(outlier)를 추가
x[9] = 1000
x

In [ ]:
x.mean(), x.std()

In [ ]:
# 1차원 -> 2차원
x = x.reshape(-1, 1)
x

In [ ]:
# sklearn에서 StandardScaler
from sklearn.preprocessing import StandardScaler

# 1. StandardScaler를 생성
standard_sclaer = StandardScaler()

# 2. fit_transform
standard_scaled = standard_sclaer.fit_transform(x)
standard_scaled

In [ ]:
# 실제로 평균이 0이고, 표준 편차가 1이 되었는가? 확인
# 4x10^-17 = 거의 0, 표준 편차 1
standard_scaled.mean(), standard_scaled.std()